# Stack Overflow Search Engine

# Problem and Data insights

### Task : Given a query deliver the most relevant or similar answers to that query

### Using the data from :https://archive.org/download/stackexchange

### Data Form : XML Files

### features: 
* Title: It contains the Title of the Post
* Question_body : It contains the Question of the Post
* Answer_body : It contains the answer of the Post

### Methodology
#### We will be using the Elastic Search Which will be containerized in a Docker Instance.
#### We will be executing the series of commands to pull and run the elastic search image from the docker repository.
#### commands:
* docker pull docker.elastic.co/elasticsearch/elasticsearch:7.9.3
* docker run -p 9200:9200 -p 9200:9300 -e "discovery.type=single-node" docker.elastic.co/elasticsearch/elasticsearch:7.9.3

### Importing Some Libraries

In [4]:
import time
import sys
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import xml.etree.ElementTree as parser
from tqdm import tqdm
import pickle
import numpy as np
from bs4 import BeautifulSoup

In [42]:
total_text_dictionary = {}
temp = {}

In [43]:
def preprocess(q):
    cleantext = BeautifulSoup(q, "lxml").text
    return cleantext.lower()

In [44]:
def iterq(tree , cat):
    for i in tree.iter('row'):
        point_dictionary = i.attrib
        # for questions and titles
        if point_dictionary['PostTypeId'] == '1':
            
            identity = cat + "_" + point_dictionary['Id']
            title = point_dictionary['Title']
            question = point_dictionary['Body']
            
            if total_text_dictionary.get(identity, None) == None:
                total_text_dictionary[identity] = []
            total_text_dictionary[identity].append(preprocess(title))
            total_text_dictionary[identity].append(preprocess(question))
            
def itera(tree , cat):
    for i in tree.iter('row'):
        point_dictionary = i.attrib
        # for answers
        if point_dictionary['PostTypeId'] == '2':
            identity = cat + "_" + point_dictionary['ParentId']
            answers = point_dictionary['Body']
            total_text_dictionary[identity].append(preprocess(answers))

## 0.1 ) Collecting the Data from different Files.

In [45]:
tree=parser.parse('D:/data/math.meta.stackexchange.com/Posts.xml').getroot()
iterq(tree , 'math')
itera(tree , 'math')

tree=parser.parse('D:/data/ai.stackexchange.com/Posts.xml').getroot()
iterq(tree , 'ai')
itera(tree , 'ai')

tree=parser.parse('D:/data/datascience.stackexchange.com/Posts.xml').getroot()
iterq(tree , 'datascience')
itera(tree , 'datascience')

tree=parser.parse('D:/data/cs.stackexchange.com/Posts.xml').getroot()
iterq(tree , 'cs')
itera(tree , 'cs')

tree=parser.parse('D:/data/softwareengineering.stackexchange.com/Posts.xml').getroot()
iterq(tree , 'soft_eng')
itera(tree , 'soft_eng')

### 0.2) Saving the Dictionary File

In [46]:
with open('text_dictionary.pickle', 'wb') as temp_var:
    pickle.dump(total_text_dictionary, temp_var)

### 0.3) Loading the Dictionary File

In [2]:
with open('text_dictionary.pickle', 'rb') as temp_var:
    total_text_dictionary = pickle.load(temp_var)

## 1.) Modelling Using the Universal-sentence-encoder

### 1.1 ) Loading the Universal-Sentence-encoder Model

In [8]:
import tensorflow as tf
embed = tf.keras.models.load_model('universal-sentence-encoder_4')
def make_vector(query):
    embeddings = embed([query])
    vector = []
    for i in embeddings[0]:
        vector.append(float(i))
    return vector

### 1.2) Connecting to the Elastic Search Container

In [4]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
if es.ping():
    print('Connected to ES!')
else:
    print('Could not connect!')
    sys.exit()

Connected to ES!


### 1.3) Defining the Structure of the Elastic Search Database 

In [8]:
structure = {"mappings": {
      "properties": {
            "total_texts": {
              "type": "text"
            },
            "total_vectors": {
                  "type": "dense_vector",
                "dims": 512
            }
        }
    }
}
ret = es.indices.create(index='database', ignore=400, body=structure)

### 1.4) Indexing of the data into the elastic search or feeding the data to the elastic search

In [ ]:
from tqdm import tqdm

for i in tqdm(total_text_dictionary.items()):
    doc_id = i[0]
    total_text = " ".join(i[1])
    point = {"total_texts":total_text ,"total_vectors" : make_vector(total_text)}
    res = es.index(index="database", id=doc_id, body=point)

### 1.5) Defining the Search Function

In [6]:
def search(query):
    def norm_list(lis):
        scores = [x[0] for x in lis]
        ma = max(scores)
        mi = min(scores)
        for i in range(len(lis)):
            lis[i][0] = (lis[i][0] - mi)/(ma - mi)
        return lis
    
    request={
            'query':{ 'match':{"total_texts":query } }
            }

    res= es.search(index='database',body=request)
    l1 = []
    for hit in res['hits']['hits']:
        l1.append([hit['_score'] , hit['_id']])
# change the cosine similarity to euclidean distance

    query_vector = make_vector(query)
    request = {"query" : {
                "script_score" : {
                    "query" : {
                        "match_all": {}
                    },
                    "script" : {
                        "source": "cosineSimilarity(params.query_vector, 'total_vectors') + 1.0",
                        "params": {"query_vector": query_vector}
                    }
                }
             }
    }

    res= es.search(index='database',body=request)
    l2 = []
    for hit in res['hits']['hits']:
        l2.append([hit['_score'] , hit['_id']])
    
    l1 = norm_list(l1)
    l2 = norm_list(l2)
    
    # getting the weighted average score for the text search and semantics search
    temp_doc = {}
    for i in l1:
        temp_doc[i[1]]  = i[0]*2
    for i in l2:
        temp_doc[i[1]] = temp_doc.get(i[1] , 0) + i[0]*5
    
    inverse_temp_doc = [(i[1] , i[0])  for i in temp_doc.items()]
    inverse_temp_doc = sorted(inverse_temp_doc , reverse = True)
    return inverse_temp_doc[:10]

# Universal Vector Encoder Results_1


In [12]:
start = time.time()
# getting the combined search results both semantic and the text based.
answer_no = 1
for i in search(" what is red black trees ? "):
    print('-'*50 + "Answer No:" + str(answer_no) + '-'*50)
    title = total_text_dictionary[i[1]][0]
    question = total_text_dictionary[i[1]][1]
    print("title : " ,title )
    print('\n')
    print("question :" , question)
    print('\n')
    sub_answer = 1
    for i in total_text_dictionary[i[1]][2:]:
        print("subanswer " + str(sub_answer) +' : ' + i )
        sub_answer+=1
    answer_no+=1
end = time.time()

--------------------------------------------------Answer No:1--------------------------------------------------
title :  why should leaf nodes in a red-black tree be black?


question : from the property of red-black trees we know that: 

all leaves (nil) are black. (all leaves are same color as the root.)(comren et al "introduction to algorithms")


but what is the reason that we should enforce them as black, even though they're nill's? 



subanswer 1 : take a uncolored leaf node, now you can color it as either red or black. if you colored it as red then you may have chance that your immediate ancestor is also red which is contradicting(according to basic principle). if you color it as black then no problem even though the immediate ancestor is red. and also no change in the number of black nodes from root to leaf paths(i.e every path get +1). this may be the possible reason behind that.

subanswer 2 : it's simply a part of the definition of a red-black tree. it is also necessary to 




subanswer 1 : it depends what you mean by isomorphic. indeed a black node with its (zero, one or two) children corresponds to a 2-3-4 node (with 1,2,3 keys and 2,3,4 children). the basic operations on the two types of trees are really different but translate quite well. 
there is a single difference in symmetry: there is only one type of 3-node (2 keys, 3 children), whereas in a red-black tree the red child to the black node may be at the left or right side.

--------------------------------------------------Answer No:8--------------------------------------------------
title :  why are red-black trees so popular?


question : it seems that everywhere i look, data structures are being implemented using red-black trees (std::set in c++, sorteddictionary in c#, etc.)
having just covered (a,b), red-black & avl trees in my algorithms class, here's what i got out (also from asking around professors, looking through a few books and googling a bit):

avl trees have smaller average depth tha


--------------------------------------------------Answer No:9--------------------------------------------------
title :  traversals from the root in avl trees and red black trees


question : we all know that for insertion() operation in avl tree following can happen:
we traverse down the tree from root to appropriate node and there insert the key and then for maintaining height balance we have to check heights of the ancestors of the newly inserted node and in doing so, we could end up traversing up to the root.
i completely agree with this.
but according to me the same can happen in red black tree because first we would traverse down the tree to appropriate node and then insert the key.then there is a possibility that a series of rotation and flip color operations could make us traverse the path up to the root.
now my question is : why following statement is right?

in avl tree insert() operation, we first traverse from root to newly inserted node and then from newly inserted node t

# Universal Vector Encoder Results_2

In [13]:
start = time.time()
# getting the combined search results both semantic and the text based.
answer_no = 1
for i in search(" what is hashmap ? "):
    print('-'*50 + "Answer No:" + str(answer_no) + '-'*50)
    title = total_text_dictionary[i[1]][0]
    question = total_text_dictionary[i[1]][1]
    print("title : " ,title )
    print('\n')
    print("question :" , question)
    print('\n')
    sub_answer = 1
    for i in total_text_dictionary[i[1]][2:]:
        print("subanswer " + str(sub_answer) +' : ' + i )
        sub_answer+=1
    answer_no+=1
end = time.time()

--------------------------------------------------Answer No:1--------------------------------------------------
title :  how hash-table and hash-map are different?


question : in the context of cs, how the hash-table and hash-map are different? 
i was watching a part of "algorithm with swift" video in udacity, and i discovered the terms "hash-table" and "hash-map" are somewhat confusing.
as far as i understand, 

hash table → stores keys only → swift set.
hash map → stores key/value pairs → swift dictionary.

but, wikipedia is talking hash-table is same thing with hash-map. here i could not find any help.
is this formal or widely accepted cs term? am i understanding correctly? if i'm wrong, please correct me. thanks.

i know industry uses their own terms that can be different, and i am asking about academia because i believe terms are more consistent in academia. if it's not, i just would abandon to learn academic terms.



subanswer 1 : if you ask me, you're correct and wikipedia is 


subanswer 2 : there's at least two points that should be raised here: 

insert and query for a hash table are not in general constant time operations in the worst case. 
there is typically no implementation that would be suitable for every possible case (i.e., different algorithms might benefit from different implementations of the graph). as an example, it doesn't seem clear how you would efficiently iterate over the neighbors of a vertex which several graph algorithms need.


--------------------------------------------------Answer No:4--------------------------------------------------
title :  can you insert into sorted list with time o(1)?


question : wondering if this was possible. if i have a sorted list, can i find the right spot for an integer and insert it, all in o(1) time? the only way i can think to do this is via having a massive hashmap with one slot for each possible integer. 2 billion different integers * 4 bytes per integer = 8 gigabytes....but it'd technically work.


--------------------------------------------------Answer No:8--------------------------------------------------
title :  why define a java object using interface (e.g. map) rather than implementation (hashmap)


question : in most java code, i see people declare java objects like this:
map<string, string> hashmap = new hashmap<>();
list<string> list = new arraylist<>();

instead of:
hashmap<string, string> hashmap = new hashmap<>();
arraylist<string> list = new arraylist<>();

why is there a preference to define the java object using the interface rather than the implementation that is actually going to be used?



subanswer 1 : having the variable constrained to an interface ensures none of the usages of that variable will be using hashmap specific functionality that may not exist on the interface, so the instance may be changed without concern later to a different implementation so long as the new instance also implements the interface.
for this reason, anytime you want to use an ob




subanswer 1 : you have to use the right tool for the right problem. mvc is used to separate the representation logic from routing request logic (controller) and business logic (the model).
for creating objects you should use creational patterns. now, in my opinion,

if i initialize the xml attributes (reading and converting to class
  objects), i'm effectively doing things in the model class that the
  controller should be doing, which is definitely not good.

it's good if the classes are used by the model.

if i put the initialization method in a controller class, i would have
  to refer to a controller class from a model class, which does not
  conform to the mvc design.

yep, don't do this because a low lever layer should not depend of a higher layer.

if i put the whole singleton in controller, then i'd have to look for
  the hashmap in the controller package, which defeats the purpose of
  having model classes.

this singleton is not a controller so don't do that too.

i can't 

# Universal Vector Encoder Results_3

In [14]:
start = time.time()
# getting the combined search results both semantic and the text based.
answer_no = 1
for i in search(" what is a linked list ? "):
    print('-'*50 + "Answer No:" + str(answer_no) + '-'*50)
    title = total_text_dictionary[i[1]][0]
    question = total_text_dictionary[i[1]][1]
    print("title : " ,title )
    print('\n')
    print("question :" , question)
    print('\n')
    sub_answer = 1
    for i in total_text_dictionary[i[1]][2:]:
        print("subanswer " + str(sub_answer) +' : ' + i )
        sub_answer+=1
    answer_no+=1
end = time.time()

--------------------------------------------------Answer No:1--------------------------------------------------
title :  finding an element in a infinite unsorted doubly linked list


question : is there any way to find an element in an unsorted doubly linked list given an element and pointer from which we can navigate? (we can't use head/tail pointers since the list is infinite)



subanswer 1 : infinite/unbounded in both directions?
zig-zag.

--------------------------------------------------Answer No:2--------------------------------------------------
title :  a question about linked list


question : i would like to check the answer of this exercise:
a circular doubled linked list with n elements has pointers that cost k bytes each of space in memory. how many bytes do the pointers of this list cost in total?
researching about circular linked lists, i can assume that each node has 2 pointers. so am i right to say that the answer is 2(n)k bytes?
thanks.



subanswer 1 : the list tak

--------------------------------------------------Answer No:10--------------------------------------------------
title :  question regarding linkedlist in java


question : when i was reading a book for scjp, i came across the following paragraph.

a linkedlist is ordered by index position, like arraylist, except that
  the elements are doubly-linked to one another. this linkage gives you
  new methods (beyond what you get from the list interface) for adding
  and removing from the beginning or end, which makes it an easy choice
  for implementing a stack or queue. keep in mind that a linkedlist may
  iterate more slowly than an arraylist, but it's a good choice when you
  need fast insertion and deletion..

what makes a linkedlist to iterate more slowly than an arraylist ?



subanswer 1 : here is an answer from stackoverflow that explains it pretty well:
https://stackoverflow.com/questions/716597/array-or-list-in-java-which-is-faster
basically, the arraylist is contiguous where the l

In [54]:
print("Time Taken:" , end - start)

Time Taken: 0.315143346786499


# 2) Modelling Using the Bert


### 2.1 ) Loading the Bert Model

In [ ]:
# using bert model
import tensorflow as tf
import tokenization
import tensorflow_hub as hub

max_seq_length = 128  # Your choice here.
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,name="segment_ids")
bert_layer = hub.KerasLayer("bert_en_uncased_L-24_H-1024_A-16_2")

pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)
from tensorflow.keras.models import Model
bert_model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=pooled_output)

def make_vector_bert(query):
    def make_mask(tokens):
        mask = []
        for i in tokens:
            if i != '[PAD]':
                mask.append(1)
            else:
                mask.append(0)
        return np.array(mask)
    
    tokens = tokenizer.tokenize(query)
    if len(tokens) < 126:
        tokens = [ '[CLS]' , *tokens ,  '[SEP]' , *['[PAD]']*(126 - len(tokens)) ]
    else:
        tokens = tokens[:126]
        tokens = ['[CLS]' , *tokens , '[SEP]']
    text = [np.array(tokenizer.convert_tokens_to_ids(tokens))]
    segment = np.array([[0]*128])
    mask =   [make_mask(tokens)]
    embeddings = bert_model.predict([text , mask , segment])
    vector = []
    for i in embeddings[0]:
        vector.append(float(i))
    return vector

### 2.2 ) Defining the Structure of the Elastic Search Database

In [4]:
# code reference from the elastic search documentation 

structure = {"mappings": {
      "properties": {
            "total_texts": {
              "type": "text"
            },
            "total_vectors": {
                  "type": "dense_vector",
                "dims": 1024
            }
        }
    }
}
ret = es.indices.create(index='database_bert', ignore=400, body=structure)

### 2.3 ) Conneting to the Elastic Search

In [15]:
# code reference from the elastic search documentation 
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
if es.ping():
    print('Connected to ES!')
else:
    print('Could not connect!')
    sys.exit()

Connected to ES!


### 2.4) Indexing of the data into the elastic search or feeding the data to the elastic search

In [ ]:
# generating the bert enbedings:
points = []
for i in tqdm(total_text_dictionary.items()):
    doc_id = i[0]
    total_text = " ".join(i[1])
    point = {"total_texts":total_text ,"total_vectors" :make_vector_bert(total_text)
    points.append(doc_id , point)

In [ ]:
# saving the dictionary file
with open('points_list.pickle', 'wb') as t:
    pickle.dump(points,t)

In [2]:
with open('points_list.pickle', 'rb') as t:
    points = pickle.load(t)

In [1]:
# indexing of the data into the elastic search or feeding the data to the elastic search
# inserting all the bert embeddings:
for i in points:
    doc_id , point = i
    res = es.index(index="database_bert", id=doc_id, body=point)
print("Completed...")

Completed...


### 2.5) Defining the Search Function

In [ ]:
# code reference from the elastic search documentation 
def search(query):

    request={
            'query':{ 'match':{"total_texts":query } }
            }

    res= es.search(index='database_bert',body=request)
    l1 = []
    for hit in res['hits']['hits']:
        l1.append([hit['_score'] , hit['_id']])
# change the cosine similarity to euclidean distance

    query_vector = make_vector_bert(query)
    request = {"query" : {
                "script_score" : {
                    "query" : {
                        "match_all": {}
                    },
                    "script" : {
                        "source": "cosineSimilarity(params.query_vector, 'total_vectors') + 1.0",
                        "params": {"query_vector": query_vector}
                    }
                }
             }
    }

    res= es.search(index='database_bert',body=request)
    l2 = []
    for hit in res['hits']['hits']:
        l2.append([hit['_score'] , hit['_id']])
    
    l1 = norm_list(l1)
    l2 = norm_list(l2)
    
    # getting the weighted average score for the text search and semantics search
    temp_doc = {}
    for i in l1:
        temp_doc[i[1]]  = i[0]*2
    for i in l2:
        temp_doc[i[1]] = temp_doc.get(i[1] , 0) + i[0]*5
    
    inverse_temp_doc = [(i[1] , i[0])  for i in temp_doc.items()]
    inverse_temp_doc = sorted(inverse_temp_doc , reverse = True)
    return inverse_temp_doc[:10]

# Bert Embeddings Results_1

In [17]:
start = time.time()
# getting the combined search results both semantic and the text based.
answer_no = 1
for i in search("what is red black trees"):
    print('-'*50 + "Answer No:" + str(answer_no) + '-'*50)
    title = total_text_dictionary[i[1]][0]
    question = total_text_dictionary[i[1]][1]
    print("title : " ,title )
    print('\n')
    print("question :" , question)
    print('\n')
    sub_answer = 1
    for i in total_text_dictionary[i[1]][2:]:
        print("subanswer " + str(sub_answer) +' : ' + i )
        sub_answer+=1
    answer_no+=1
end = time.time()

--------------------------------------------------Answer No:1--------------------------------------------------
title :  why should leaf nodes in a red-black tree be black?


question : from the property of red-black trees we know that: 

all leaves (nil) are black. (all leaves are same color as the root.)(comren et al "introduction to algorithms")


but what is the reason that we should enforce them as black, even though they're nill's? 



subanswer 1 : take a uncolored leaf node, now you can color it as either red or black. if you colored it as red then you may have chance that your immediate ancestor is also red which is contradicting(according to basic principle). if you color it as black then no problem even though the immediate ancestor is red. and also no change in the number of black nodes from root to leaf paths(i.e every path get +1). this may be the possible reason behind that.

subanswer 2 : it's simply a part of the definition of a red-black tree. it is also necessary to 




subanswer 1 : it depends what you mean by isomorphic. indeed a black node with its (zero, one or two) children corresponds to a 2-3-4 node (with 1,2,3 keys and 2,3,4 children). the basic operations on the two types of trees are really different but translate quite well. 
there is a single difference in symmetry: there is only one type of 3-node (2 keys, 3 children), whereas in a red-black tree the red child to the black node may be at the left or right side.

--------------------------------------------------Answer No:8--------------------------------------------------
title :  why are red-black trees so popular?


question : it seems that everywhere i look, data structures are being implemented using red-black trees (std::set in c++, sorteddictionary in c#, etc.)
having just covered (a,b), red-black & avl trees in my algorithms class, here's what i got out (also from asking around professors, looking through a few books and googling a bit):

avl trees have smaller average depth tha


--------------------------------------------------Answer No:9--------------------------------------------------
title :  traversals from the root in avl trees and red black trees


question : we all know that for insertion() operation in avl tree following can happen:
we traverse down the tree from root to appropriate node and there insert the key and then for maintaining height balance we have to check heights of the ancestors of the newly inserted node and in doing so, we could end up traversing up to the root.
i completely agree with this.
but according to me the same can happen in red black tree because first we would traverse down the tree to appropriate node and then insert the key.then there is a possibility that a series of rotation and flip color operations could make us traverse the path up to the root.
now my question is : why following statement is right?

in avl tree insert() operation, we first traverse from root to newly inserted node and then from newly inserted node t

# Bert Embeddings Results_2

In [18]:
start = time.time()
# getting the combined search results both semantic and the text based.
answer_no = 1
for i in search("what is hashmap"):
    print('-'*50 + "Answer No:" + str(answer_no) + '-'*50)
    title = total_text_dictionary[i[1]][0]
    question = total_text_dictionary[i[1]][1]
    print("title : " ,title )
    print('\n')
    print("question :" , question)
    print('\n')
    sub_answer = 1
    for i in total_text_dictionary[i[1]][2:]:
        print("subanswer " + str(sub_answer) +' : ' + i )
        sub_answer+=1
    answer_no+=1
end = time.time()

--------------------------------------------------Answer No:1--------------------------------------------------
title :  how hash-table and hash-map are different?


question : in the context of cs, how the hash-table and hash-map are different? 
i was watching a part of "algorithm with swift" video in udacity, and i discovered the terms "hash-table" and "hash-map" are somewhat confusing.
as far as i understand, 

hash table → stores keys only → swift set.
hash map → stores key/value pairs → swift dictionary.

but, wikipedia is talking hash-table is same thing with hash-map. here i could not find any help.
is this formal or widely accepted cs term? am i understanding correctly? if i'm wrong, please correct me. thanks.

i know industry uses their own terms that can be different, and i am asking about academia because i believe terms are more consistent in academia. if it's not, i just would abandon to learn academic terms.



subanswer 1 : if you ask me, you're correct and wikipedia is 


subanswer 2 : there's at least two points that should be raised here: 

insert and query for a hash table are not in general constant time operations in the worst case. 
there is typically no implementation that would be suitable for every possible case (i.e., different algorithms might benefit from different implementations of the graph). as an example, it doesn't seem clear how you would efficiently iterate over the neighbors of a vertex which several graph algorithms need.


--------------------------------------------------Answer No:4--------------------------------------------------
title :  can you insert into sorted list with time o(1)?


question : wondering if this was possible. if i have a sorted list, can i find the right spot for an integer and insert it, all in o(1) time? the only way i can think to do this is via having a massive hashmap with one slot for each possible integer. 2 billion different integers * 4 bytes per integer = 8 gigabytes....but it'd technically work.


--------------------------------------------------Answer No:8--------------------------------------------------
title :  why define a java object using interface (e.g. map) rather than implementation (hashmap)


question : in most java code, i see people declare java objects like this:
map<string, string> hashmap = new hashmap<>();
list<string> list = new arraylist<>();

instead of:
hashmap<string, string> hashmap = new hashmap<>();
arraylist<string> list = new arraylist<>();

why is there a preference to define the java object using the interface rather than the implementation that is actually going to be used?



subanswer 1 : having the variable constrained to an interface ensures none of the usages of that variable will be using hashmap specific functionality that may not exist on the interface, so the instance may be changed without concern later to a different implementation so long as the new instance also implements the interface.
for this reason, anytime you want to use an ob




subanswer 1 : you have to use the right tool for the right problem. mvc is used to separate the representation logic from routing request logic (controller) and business logic (the model).
for creating objects you should use creational patterns. now, in my opinion,

if i initialize the xml attributes (reading and converting to class
  objects), i'm effectively doing things in the model class that the
  controller should be doing, which is definitely not good.

it's good if the classes are used by the model.

if i put the initialization method in a controller class, i would have
  to refer to a controller class from a model class, which does not
  conform to the mvc design.

yep, don't do this because a low lever layer should not depend of a higher layer.

if i put the whole singleton in controller, then i'd have to look for
  the hashmap in the controller package, which defeats the purpose of
  having model classes.

this singleton is not a controller so don't do that too.

i can't 

# Bert Embeddings Results_3

In [20]:
start = time.time()
# getting the combined search results both semantic and the text based.
answer_no = 1
for i in search("what is a linked list"):
    print('-'*50 + "Answer No:" + str(answer_no) + '-'*50)
    title = total_text_dictionary[i[1]][0]
    question = total_text_dictionary[i[1]][1]
    print("title : " ,title )
    print('\n')
    print("question :" , question)
    print('\n')
    sub_answer = 1
    for i in total_text_dictionary[i[1]][2:]:
        print("subanswer " + str(sub_answer) +' : ' + i )
        sub_answer+=1
    answer_no+=1
end = time.time()

--------------------------------------------------Answer No:1--------------------------------------------------
title :  finding an element in a infinite unsorted doubly linked list


question : is there any way to find an element in an unsorted doubly linked list given an element and pointer from which we can navigate? (we can't use head/tail pointers since the list is infinite)



subanswer 1 : infinite/unbounded in both directions?
zig-zag.

--------------------------------------------------Answer No:2--------------------------------------------------
title :  a question about linked list


question : i would like to check the answer of this exercise:
a circular doubled linked list with n elements has pointers that cost k bytes each of space in memory. how many bytes do the pointers of this list cost in total?
researching about circular linked lists, i can assume that each node has 2 pointers. so am i right to say that the answer is 2(n)k bytes?
thanks.



subanswer 1 : the list tak

--------------------------------------------------Answer No:10--------------------------------------------------
title :  question regarding linkedlist in java


question : when i was reading a book for scjp, i came across the following paragraph.

a linkedlist is ordered by index position, like arraylist, except that
  the elements are doubly-linked to one another. this linkage gives you
  new methods (beyond what you get from the list interface) for adding
  and removing from the beginning or end, which makes it an easy choice
  for implementing a stack or queue. keep in mind that a linkedlist may
  iterate more slowly than an arraylist, but it's a good choice when you
  need fast insertion and deletion..

what makes a linkedlist to iterate more slowly than an arraylist ?



subanswer 1 : here is an answer from stackoverflow that explains it pretty well:
https://stackoverflow.com/questions/716597/array-or-list-in-java-which-is-faster
basically, the arraylist is contiguous where the l